In [1]:
import pandas as pd
import dask.dataframe as df
from dask_ml.preprocessing import OneHotEncoder
import numpy as np
from keras.utils.np_utils import to_categorical
from dask.distributed import Client
import time
import json
import os

from sklearn.model_selection import train_test_split
from typing import Callable, List
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from dask import array as da
from textacy.preprocess import preprocess_text
import dask.multiprocessing
from pathos.multiprocessing import cpu_count
from collections import Counter
from collections import defaultdict
import h5py

Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
client = Client(os.getenv("DASK_SCHEDULER_ADDRESS"))

In [3]:
start_time = time.time()

output_dir = "/data/"

base_url = 'https://storage.googleapis.com/codenet/issue_labels/'
dd = df.from_pandas(pd.concat([pd.read_csv(base_url+f'00000000000{i}.csv.gz') for i in range(1)]), npartitions=64)

def textacy_cleaner(text: str) -> str:
    """a
    Defines the default function for cleaning text.

    This function operates over a list.
    """
    return preprocess_text(text,
                           lowercase=True,
                           no_emails=True,
                           no_phone_numbers=True,
                           no_numbers=True,
                           no_currency_symbols=True,
                           no_punct=True,
                           no_contractions=False,
                           no_accents=True)


def process_document(doc: str) -> List[str]:
    if doc and len(doc) > 20000:
        return ["_start_", "", "_end_"]
    doc = text_to_word_sequence(textacy_cleaner(doc))
    if len(doc) > 1000:
        return ["_start_", "", "_end_"]
    return ["_start_"] + doc + ["_end_"]


test_data = 'hello world 314-903-3072, hamel.husain@gmail.com wee woo'
assert process_document(test_data) == ['_start_', 'hello', 'world', 'phone', 'email', 'wee', 'woo', '_end_']


bodies_parsed = dd["body"].apply(process_document)
titles_parsed = dd["title"].apply(process_document)

/usr/local/lib/python3.6/site-packages/dask/dataframe/core.py:2966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('body', 'object'))

  warnings.warn(meta_warning(meta))
/usr/local/lib/python3.6/site-packages/dask/dataframe/core.py:2966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('title', 'object'))

  warnings.warn(meta_wa

In [4]:
now = time.time() - start_time
print(f"tokenized {now}")

def to_one_hot(df):
    return to_categorical(df.values, num_classes=3)

targets = dd["class_int"].to_frame().map_partitions(to_one_hot)

body_quant = int(bodies_parsed.apply(len).quantile(q=0.85).compute())
title_quant = int(titles_parsed.apply(len).quantile(q=0.85).compute())

print(f"Quantiles title-{title_quant} body-{body_quant} ")

def drop_long_docs(doc, max_len):
    if len(doc) > max_len:
        return doc[:max_len]
    return doc

bodies_parsed = bodies_parsed.apply(drop_long_docs, max_len=body_quant)
titles_parsed = titles_parsed.apply(drop_long_docs, max_len=title_quant)

def count_words(partition):
    c = Counter()
    def count(p):
        c.update(p)
        return c
    ct = Counter()
    ct.update(dict(partition.apply(count).iloc[0].most_common(n=8000)))
    return ct


now = time.time() - start_time
print(f"quantiles done {now}")

/usr/local/lib/python3.6/site-packages/dask/dataframe/core.py:2966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('body', 'int64'))

  warnings.warn(meta_warning(meta))


tokenized 8.895066499710083


/usr/local/lib/python3.6/site-packages/dask/dataframe/core.py:2966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('title', 'int64'))

  warnings.warn(meta_warning(meta))


Quantiles title-13 body-172 
quantiles done 123.69262671470642


/usr/local/lib/python3.6/site-packages/dask/dataframe/core.py:2966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('body', 'object'))

  warnings.warn(meta_warning(meta))
/usr/local/lib/python3.6/site-packages/dask/dataframe/core.py:2966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('title', 'object'))

  warnings.warn(meta_wa

In [5]:
body_counts = bodies_parsed.map_partitions(count_words).compute()
now = time.time() - start_time
print(f"body counts computed {now}")
body_counts = sum(body_counts.tolist(), Counter())
now = time.time() - start_time
print(f"body-counts done {now}")
title_counts = titles_parsed.map_partitions(count_words).compute()
title_counts = sum(title_counts.tolist(), Counter())

now = time.time() - start_time
print(f"counting words body {now}")

words_to_keep_body = body_counts.most_common(n=8000)
body_vocab = defaultdict(lambda: 1)
body_vocab.update({x:i+2 for i, x in enumerate([x[0] for x in words_to_keep_body])})

/usr/local/lib/python3.6/site-packages/distributed/worker.py:3217: UserWarning: Large object of size 3.73 MB detected in task graph: 
  ('apply-628d5b10edef41eb4b38b63939b9371c', <functi ... lumns], 'body')
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


body counts computed 224.82254028320312
body-counts done 227.7306730747223
counting words body 250.6220347881317


In [6]:
now = time.time() - start_time
print(f"counting words title {now}")
words_to_keep_title = title_counts.most_common(n=4500)
titles_vocab = defaultdict(lambda: 1)
titles_vocab.update({x:i+2 for i, x in enumerate([x[0] for x in words_to_keep_title])})

now = time.time() - start_time
print(f"words counted {now}")

numer_bodies = bodies_parsed.apply(lambda x: [body_vocab[w] for w in x])
numer_titles = titles_parsed.apply(lambda x: [titles_vocab[w] for w in x])

def pad_partition(numerized_doc, max_len):
    if type(numerized_doc) != list:
        return
    return pad_sequences([numerized_doc], maxlen=max_len, truncating='post')[0]

processed_bodies = numer_bodies.apply(pad_partition, max_len=body_quant)
processed_titles = numer_titles.apply(pad_partition, max_len=title_quant)

now = time.time() - start_time
print(f"saving {now}")

processed_titles = np.stack(processed_titles.values.compute())
processed_bodies = np.stack(processed_bodies.values.compute())

counting words title 250.68553686141968
words counted 250.7349832057953
saving 250.87703490257263


/usr/local/lib/python3.6/site-packages/dask/dataframe/core.py:2966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('body', 'object'))

  warnings.warn(meta_warning(meta))
/usr/local/lib/python3.6/site-packages/dask/dataframe/core.py:2966: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('title', 'object'))

  warnings.warn(meta_wa

In [7]:
now = time.time() - start_time
print(f"creating hdf5 {now}")


f = h5py.File('/data/dataset.hdf5', 'w')
f.create_dataset('/titles', data=processed_titles)
f.create_dataset('/bodies', data=processed_bodies)
f.create_dataset('/targets', data=targets)
f.close()

with open("/data/metadata.json", "w") as f:
    meta = {
        'body_vocab_size': len(body_vocab),
        'title_vocab_size': len(titles_vocab),
        'issue_body_doc_length': body_quant,
        'issue_title_doc_length': title_quant,
    }
    f.write(json.dumps(meta))

creating hdf5 376.64964151382446


In [8]:
now = time.time() - start_time
print(f"saved {now}")

saved 382.772980928421
